# atmodeller

## Tutorial 3: Monte Carlo experiment

We can devise a simple Monte Carlo approach to sample the probable atmospheres that can arise for different planetary conditions.

Import the required packages and set the package logger to the INFO level. For more output you could instead set it to DEBUG.

In [1]:
from atmodeller import InteriorAtmosphereSystem, Molecule, Planet, OCEAN_MOLES, SystemConstraint, logger, MassConstraint, BufferedFugacityConstraint, MolarMasses
from atmodeller.thermodynamics import PeridotiteH2O, NoSolubility, BasaltDixonCO2, BasaltLibourelN2
import numpy as np
import csv
import logging

logger.setLevel(logging.INFO)

15:00:40 - atmodeller                     - INFO      - atmodeller version 0.1.0


In [2]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='N2', solubility=BasaltLibourelN2(), solid_melt_distribution_coefficient=0))
molecules

15:00:41 - atmodeller.thermodynamics      - INFO      - Creating a molecule: H2O
15:00:41 - atmodeller.thermodynamics      - INFO      - Creating a molecule: H2
15:00:41 - atmodeller.thermodynamics      - INFO      - Creating a molecule: O2
15:00:41 - atmodeller.thermodynamics      - INFO      - Creating a molecule: CO
15:00:41 - atmodeller.thermodynamics      - INFO      - Creating a molecule: CO2
15:00:41 - atmodeller.thermodynamics      - INFO      - Creating a molecule: N2


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x1292a37d0>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, molar_mass=0.018015200000000002, output=None),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x11f32bd50>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, molar_mass=0.0020158, output=None),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x1208594d0>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, molar_mass=0.0319988, output=None),
 Molecule(name='CO', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x10ec0e090>, solid_melt_distribution_coefficient=0, elements={'C': 1, 'O': 1}, element_masses={'C': 0.0120107, 'O': 0.0159994}, molar_mass=0.0280101, output=None),
 Molecule(name='CO2', solubility=<atmodeller.th

In [3]:
planet: Planet = Planet()
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules, planet=planet)

15:00:41 - atmodeller.thermodynamics      - INFO      - Creating a new planet
15:00:41 - atmodeller.thermodynamics      - INFO      - Mantle mass (kg) = 4208261222595110885130240.000000
15:00:41 - atmodeller.thermodynamics      - INFO      - Mantle melt fraction = 1.000000
15:00:41 - atmodeller.thermodynamics      - INFO      - Core mass fraction = 0.295335
15:00:41 - atmodeller.thermodynamics      - INFO      - Planetary radius (m) = 6371000.000000
15:00:41 - atmodeller.thermodynamics      - INFO      - Planetary mass (kg) = 5972000000000000327155712.000000
15:00:41 - atmodeller.thermodynamics      - INFO      - Surface temperature (K) = 2000.000000
15:00:41 - atmodeller.thermodynamics      - INFO      - Surface gravity (m/s^2) = 9.819973
15:00:41 - atmodeller.thermodynamics      - INFO      - Melt Composition = None
15:00:41 - atmodeller.core                - INFO      - Creating a new interior-atmosphere system
15:00:41 - atmodeller.core                - INFO      - Molecules = ['CO

In [4]:
number_of_realisations: int = 2

# Parameters are normally distributed between bounds.
number_ocean_moles: np.ndarray = np.random.uniform(1, 10, number_of_realisations)
ch_ratios: np.ndarray = np.random.uniform(0.1, 1, number_of_realisations)
fo2_shifts: np.ndarray = np.random.uniform(-4, 4, number_of_realisations)

# Store the output in a list.
out: list[dict[str, float]] = []

# ppmw of Nitrogen in the mantle. 2.8 is the mantle value of N.
N_ppmw: float = 2.8

# The nitrogen mass is constant.
mass_N: float = N_ppmw * 1.0e-6 * planet.mantle_mass

for realisation in range(number_of_realisations):

    mass_H: float = number_ocean_moles[realisation] * OCEAN_MOLES * MolarMasses().H2
    mass_C: float = ch_ratios[realisation] * mass_H
    constraints: list[SystemConstraint] = [
        MassConstraint(species="H", value=mass_H),
        MassConstraint(species="C", value=mass_C),
        MassConstraint(species="N", value=mass_N),
        BufferedFugacityConstraint(log10_shift=fo2_shifts[realisation])
    ]
    # Recall that changing attributes on the planet 'object' will be 'seen' by interior_atmosphere.
    out_realisation = interior_atmosphere.solve(constraints)

    # Include the parameters in the output.
    out_realisation['number_ocean_moles'] = number_ocean_moles[realisation]
    out_realisation['ch_ratio'] = ch_ratios[realisation]
    out_realisation['fo2_shift'] = fo2_shifts[realisation]

    out.append(out_realisation)

    filename: str = "atmodeller_monte_carlo_tutorial3.csv"
    print("Writing output to: %s", filename)
    fieldnames: list[str] = list(out[0].keys())
    with open(filename, "w", newline="", encoding="utf-8") as csvfile:
        writer: csv.DictWriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(out)

15:00:41 - atmodeller.core                - INFO      - Constraints: [MassConstraint(species='H', value=1.5340869325596278e+21, field='mass'),
 MassConstraint(species='C', value=7.421288326236803e+20, field='mass'),
 MassConstraint(species='N', value=1.178313142326631e+19, field='mass'),
 BufferedFugacityConstraint(species='O2',
                            fugacity=<atmodeller.thermodynamics.IronWustiteBufferOneill object at 0x11f363290>,
                            log10_shift=2.8296662948708375,
                            field='fugacity')]
15:00:41 - atmodeller.core                - INFO      - Mixed pressure and mass constraints so attempting to solve a non-linear system of equations
15:00:41 - atmodeller.core                - INFO      - 3 additional (not fugacity) constraint(s) are necessary to solve the system
15:00:41 - atmodeller.core                - INFO      - Row 00: Reaction 0: 1.0 CO + 0.5 O2 = 1.0 CO2
15:00:41 - atmodeller.core                - INFO      - Row 01: Reac

Writing output to: %s atmodeller_monte_carlo_tutorial3.csv
Writing output to: %s atmodeller_monte_carlo_tutorial3.csv


The simulation data is output as:

In [5]:
out

[{'CO': 73.91632148711004,
  'H2': 2.1495545732610934,
  'N2': 2.9093566838232854,
  'O2': 2.6417355053613478e-05,
  'CO2': 291.9482088699299,
  'H2O': 38.104155583354995,
  'number_ocean_moles': 9.897727839129914,
  'ch_ratio': 0.4837593078153899,
  'fo2_shift': 2.8296662948708375},
 {'CO': 97.3695376383669,
  'H2': 0.9937452226890828,
  'N2': 2.9374879574343886,
  'O2': 1.1997360884252258e-05,
  'CO2': 259.17121934375916,
  'H2O': 11.87126814607474,
  'number_ocean_moles': 5.50742022036262,
  'ch_ratio': 0.8260242244812641,
  'fo2_shift': 2.486862684409301}]